### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [20]:
# Start by finding amount of schools and students, force that to be an integer for ease of use
totalSchools = int(len(pd.unique(school_data_complete["school_name"])))
totalStudents = int(len(pd.unique(school_data_complete["Student ID"])))


# Sum the data from the "budget" category
totalBudget = school_data["budget"].sum()


# Calculate the averages using total students and sum of scores
averageMathScore = (int(student_data["math_score"].sum()) / totalStudents)
averageReadingScore = (int(student_data["reading_score"].sum()) / totalStudents)


# Calculate percentage of students who passed math or reading
passingMath = (100*(len(student_data.loc[student_data["math_score"] >= 70, ["Student ID"]])/totalStudents))
passingReading = (100*(len(student_data.loc[student_data["reading_score"] >= 70, ["Student ID"]])/totalStudents))


# Have to find overlap of students with >= 70 in both math and reading, easy two-step process:
passedMath=student_data.loc[student_data["math_score"] >= 70, ["Student ID", "reading_score"]]
overallPassing = (100*(int(len(passedMath.loc[passedMath["reading_score"] >= 70, ["Student ID"]]))/totalStudents))

# Put it all together
summary_df = pd.DataFrame({"Total Schools": [totalSchools],
                            "Total Students": [totalStudents],
                            "Total Budget": totalBudget,
                            "Average Math Score": averageMathScore,
                            "Average Reading Score": averageReadingScore,
                            "% Passing Math": passingMath,
                            "% Passing Reading": passingReading,
                            "% Overall Passing": overallPassing
                          })


# Formatting
summary_df["Total Budget"] = summary_df["Total Budget"].map("${:,.2f}".format)
summary_df["Total Students"] = summary_df["Total Students"].map("{:,}".format)


# Display result
summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [23]:

# Get each school's size
totalStudentsList = school_data[["school_name", "size"]]


# Get each school's type
schoolTypeList = school_data[["school_name", "type"]]


# Get each school's budget
schoolBudgetList = school_data[["school_name", "budget"]]


# Merge the first three categories into one dataframe
school_summary_df = pd.merge(schoolTypeList, totalStudentsList, on="school_name")
school_summary_df = pd.merge(school_summary_df, schoolBudgetList, on="school_name")


# Divide the budget column by the size column to get budget per student
school_summary_df = school_summary_df.assign(PerStudentBudget= (school_summary_df["budget"]/school_summary_df["size"] ))


# Used in a later cell
budgetPerStudent = school_summary_df.assign(PerStudentBudget= (school_summary_df["budget"]/school_summary_df["size"] ))


# Count sum of math/reading scores per school
mathData = student_data.groupby("school_name")["math_score"].sum()
readingData = student_data.groupby("school_name")["reading_score"].sum()


# Merge with each school's size
mathDataSize = pd.merge(mathData, totalStudentsList, on="school_name")
readingDataSize = pd.merge(readingData, totalStudentsList, on="school_name")


# Calculate average scores
mathDataAverage = mathDataSize.assign(AverageMathScore = (mathDataSize["math_score"]/mathDataSize["size"] ))
readingDataAverage = readingDataSize.assign(AverageReadingScore = (readingDataSize["reading_score"]/readingDataSize["size"] ))


# Take school name and average scores, merge that with the main dataframe (school_summary_df)
mathDataAverage = mathDataAverage[["school_name", "AverageMathScore"]]
readingDataAverage = readingDataAverage[["school_name", "AverageReadingScore"]]

school_summary_df = pd.merge(school_summary_df, mathDataAverage, on="school_name")
school_summary_df = pd.merge(school_summary_df, readingDataAverage, on="school_name")


# Take each school's student's grades 70 or above
passingMathList = student_data.loc[student_data["math_score"] >= 70, ["school_name", "math_score", "reading_score", "Student ID", "grade"]]
passingReadingList = student_data.loc[student_data["reading_score"] >= 70, ["school_name", "math_score", "reading_score", "Student ID", "grade"]]
passingBothList = passingMathList.loc[passingMathList["reading_score"] >= 70, ["school_name", "math_score", "reading_score", "Student ID", "grade"]]


# Count number passed for each school
passingMathListCount = passingMathList.groupby(["school_name"]).count()
passingReadingListCount = passingReadingList.groupby(["school_name"]).count()
passingBothListCount = passingBothList.groupby(["school_name"]).count()


# Merge and format correctly
passingMathSize = pd.merge(passingMathListCount, totalStudentsList, on="school_name")
passingMathSize = passingMathSize[["school_name","math_score"]]
passingMathSize = passingMathSize.rename(columns={"math_score":"num_passed_math"})

passingReadingSize = pd.merge(passingReadingListCount, totalStudentsList, on="school_name")
passingReadingSize = passingReadingSize[["school_name","math_score"]]
passingReadingSize = passingReadingSize.rename(columns={"math_score":"num_passed_reading"})

passedBothSize = pd.merge(passingBothListCount, totalStudentsList, on="school_name")
passedBothSize = passedBothSize[["school_name","math_score"]]
passedBothSize = passedBothSize.rename(columns={"math_score":"num_passed_both"})


# Merge with size again
passingMathSize = pd.merge(passingMathSize, totalStudentsList, on="school_name")
passingReadingSize = pd.merge(passingReadingSize, totalStudentsList, on="school_name")
passedBothSize = pd.merge(passedBothSize, totalStudentsList, on="school_name")


# Calculate passing per school and keep only that column
passingMathSize = passingMathSize.assign(PercentPassingMath = (100*passingMathSize["num_passed_math"]/passingMathSize["size"] ))
passingMathSize = passingMathSize[["PercentPassingMath","school_name"]]

passingReadingSize = passingReadingSize.assign(PercentPassingReading = (100*passingReadingSize["num_passed_reading"]/passingReadingSize["size"] ))
passingReadingSize = passingReadingSize[["PercentPassingReading","school_name"]]

passedBothSize = passedBothSize.assign(PercentPassingBoth = (100*passedBothSize["num_passed_both"]/passedBothSize["size"] ))
passedBothSize = passedBothSize[["PercentPassingBoth","school_name"]]


# Merge to main summary 
school_summary_df = pd.merge(school_summary_df, passingMathSize, on="school_name")
school_summary_df = pd.merge(school_summary_df, passingReadingSize, on="school_name")
school_summary_df = pd.merge(school_summary_df, passedBothSize, on="school_name")


school_summary_df = school_summary_df.rename(columns={"school_name":"School",
                                                      "type":"School Type",
                                                      "size":"Total Students",
                                                     "budget":"Total School Budget",
                                                     "PerStudentBudget":"Per Student Budget",
                                                     "AverageMathScore":"Average Math Score",
                                                     "AverageReadingScore":"Average Reading Score",
                                                     "PercentPassingMath":"% Passing Math",
                                                     "PercentPassingReading":"% Passing Reading",
                                                     "PercentPassingBoth":"% Overall Passing"})


# Formatting
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,}".format)


# Display final result
school_summary_df



,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
2,Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
3,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
6,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
7,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [24]:
# Using the data computed in the previous cell, show top 5 schools by overall passing
sortedByOverallPassing = school_summary_df.sort_values(
    ["% Overall Passing"], ascending=False)
sortedByOverallPassing.head(5)



,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
6,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
14,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
5,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [25]:
# Sort by showing bottom 5 schools
sortedByOverallPassing = school_summary_df.sort_values(
    ["% Overall Passing"], ascending=True)
sortedByOverallPassing.head(5)

,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
11,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
0,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
12,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [26]:
# Retrieve necessary data and duplicate columns for ease of use
mathScoreByGrade = student_data[["school_name","grade","math_score"]]
schoolAndGradeStorage = student_data[["school_name","grade"]]
schoolAndGradeStorage = schoolAndGradeStorage.assign(School = schoolAndGradeStorage["school_name"])
schoolAndGradeStorage = schoolAndGradeStorage.assign(Grade = schoolAndGradeStorage["grade"])


# Organize the data by grade and school name
mathScoreByGradeSum = mathScoreByGrade.groupby(["grade","school_name"]).sum()
mathScoreByGradeSize = mathScoreByGrade.groupby(["grade","school_name"]).count()
mathScoreByGradeSize = mathScoreByGradeSize.rename(columns={"math_score":"class_size"})


# Merge data and compute average per class
mergedSizeScore = pd.merge(mathScoreByGradeSum, mathScoreByGradeSize,  how="left", left_on=["grade","school_name"], right_on = ["grade","school_name"])
mergedSizeScore = mergedSizeScore.assign(average_math_score = (mergedSizeScore["math_score"]/mergedSizeScore["class_size"] ))
mergedSizeScore = pd.merge(mergedSizeScore, schoolAndGradeStorage,  how="left", left_on=["grade","school_name"], right_on = ["grade","school_name"])


# Get only what we need, and drop duplicates
averageScorePerClass = mergedSizeScore[["School","Grade","average_math_score"]]
averageScorePerClass = averageScorePerClass.drop_duplicates()


# Put Grade level on x-axis and high school on y-axis
math_summary_df = averageScorePerClass.pivot(index="School", columns="Grade", values="average_math_score")
math_summary_df



Grade,10th,11th,12th,9th
School,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [27]:
# Retrieve necessary data and duplicate columns for ease of use
readingScoreByGrade = student_data[["school_name","grade","reading_score"]]
schoolAndGradeStorageReading = student_data[["school_name","grade"]]
schoolAndGradeStorageReading = schoolAndGradeStorageReading.assign(School = schoolAndGradeStorageReading["school_name"])
schoolAndGradeStorageReading = schoolAndGradeStorageReading.assign(Grade = schoolAndGradeStorageReading["grade"])


# Organize the data by grade and school name
readingScoreByGradeSum = readingScoreByGrade.groupby(["grade","school_name"]).sum()
readingScoreByGradeSize = readingScoreByGrade.groupby(["grade","school_name"]).count()
readingScoreByGradeSize = readingScoreByGradeSize.rename(columns={"reading_score":"class_size"})


# Merge data and compute average per class
mergedReadingSizeScore = pd.merge(readingScoreByGradeSum, readingScoreByGradeSize,  how="left", left_on=["grade","school_name"], right_on = ["grade","school_name"])
mergedReadingSizeScore = mergedReadingSizeScore.assign(average_reading_score = (mergedReadingSizeScore["reading_score"]/mergedReadingSizeScore["class_size"] ))
mergedReadingSizeScore = pd.merge(mergedReadingSizeScore, schoolAndGradeStorageReading,  how="left", left_on=["grade","school_name"], right_on = ["grade","school_name"])


# Get only what we need and drop duplicates
averageReadingScorePerClass = mergedReadingSizeScore[["School","Grade","average_reading_score"]]
averageReadingScorePerClass = averageReadingScorePerClass.drop_duplicates()


# Put Grade level on x-axis and high school on y-axis
reading_summary_df = averageReadingScorePerClass.pivot(index="School", columns="Grade", values="average_reading_score")
reading_summary_df


Grade,10th,11th,12th,9th
School,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [28]:
# Need data on spending per student for each student, math scores for each student, reading scores for each student (sort by ID)
studentsSpendingAndScores = student_data[["Student ID", "school_name", "reading_score", "math_score"]]
budgetPerStudent = budgetPerStudent[["school_name","PerStudentBudget"]]
studentsSpendingAndScores = pd.merge(studentsSpendingAndScores, budgetPerStudent, on="school_name")


# Create bin bounds
scoreBins = [0, 585, 630, 645, 680]


# Create bin labels
group_labels = ["<$585", "$585-630", "$630-645", "$645-680"]  


# Apply the bins to the data
pd.cut(studentsSpendingAndScores["PerStudentBudget"], scoreBins, labels=group_labels).head()


# Add to the dataframe we're using
studentsSpendingAndScores["Spending Ranges (Per Student)"] = pd.cut(studentsSpendingAndScores["PerStudentBudget"], scoreBins, labels=group_labels)


# Remove unneeded data
studentsSpendingAndScores = studentsSpendingAndScores[["reading_score","math_score","Spending Ranges (Per Student)"]]


# Organize the data by the bins (to get averages)
scoreByBudgetSum = studentsSpendingAndScores.groupby(["Spending Ranges (Per Student)"]).sum()

scoreByBudgetSize = studentsSpendingAndScores.groupby(["Spending Ranges (Per Student)"]).count()
scoreByBudgetSize = scoreByBudgetSize.rename(columns={"reading_score":"reading_size","math_score":"math_size"})

AverageScoresByBudget = pd.merge(scoreByBudgetSize, scoreByBudgetSum, on="Spending Ranges (Per Student)")


# Calculate averages and keep them
AverageScoresByBudget = AverageScoresByBudget.assign(average_math_score = (AverageScoresByBudget["math_score"]/AverageScoresByBudget["math_size"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(average_reading_score = (AverageScoresByBudget["reading_score"]/AverageScoresByBudget["reading_size"] ))
AverageScoresByBudget = AverageScoresByBudget[["average_math_score","average_reading_score"]]
AverageScoresByBudget = AverageScoresByBudget.rename(columns={"average_math_score":"Average Math Score","average_reading_score":"Average Reading Score"})


# Identify who is passing each category
passingMathListByBudget = studentsSpendingAndScores.loc[studentsSpendingAndScores["math_score"] >= 70]
passingReadingListByBudget = studentsSpendingAndScores.loc[studentsSpendingAndScores["reading_score"] >= 70]
passingBothListByBudget = passingMathListByBudget.loc[passingMathListByBudget["reading_score"] >= 70]


# Sum how many students are passing in each range
passingMathListByBudget = passingMathListByBudget.groupby(["Spending Ranges (Per Student)"]).count()
passingMathListByBudget = passingMathListByBudget[["reading_score"]]
passingMathListByBudget = passingMathListByBudget.rename(columns={"reading_score":"amount_passed_math"})

passingReadingListByBudget = passingReadingListByBudget.groupby(["Spending Ranges (Per Student)"]).count()
passingReadingListByBudget = passingReadingListByBudget[["reading_score"]]
passingReadingListByBudget = passingReadingListByBudget.rename(columns={"reading_score":"amount_passed_reading"})

passingBothListByBudget = passingBothListByBudget.groupby(["Spending Ranges (Per Student)"]).count()
passingBothListByBudget = passingBothListByBudget[["reading_score"]]
passingBothListByBudget = passingBothListByBudget.rename(columns={"reading_score":"amount_passed_both"})


# Count number of students from each range
countByRange = studentsSpendingAndScores.groupby(["Spending Ranges (Per Student)"]).count()
countByRange = countByRange[["reading_score"]]
countByRange = countByRange.rename(columns={"reading_score":"total_count"})


# Add necessary information to the chart
AverageScoresByBudget = pd.merge(AverageScoresByBudget, countByRange, on="Spending Ranges (Per Student)")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingMathListByBudget, on="Spending Ranges (Per Student)")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingReadingListByBudget, on="Spending Ranges (Per Student)")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingBothListByBudget, on="Spending Ranges (Per Student)")


# Calculate percentages of students passing in new columns
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_math = (100*AverageScoresByBudget["amount_passed_math"]/AverageScoresByBudget["total_count"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_reading = (100*AverageScoresByBudget["amount_passed_reading"]/AverageScoresByBudget["total_count"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_both = (100*AverageScoresByBudget["amount_passed_both"]/AverageScoresByBudget["total_count"] ))


# Re-name new columns
AverageScoresByBudget = AverageScoresByBudget.rename(columns={"percent_passed_math":"% Passing Math","percent_passed_reading":"% Passing Reading", "percent_passed_both":"% Overall Passing"})


# Keep only what is needed
AverageScoresByBudget = AverageScoresByBudget[["Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]


# Display result
AverageScoresByBudget





,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,90.640704
$585-630,79.982873,82.312643,79.109851,88.513145,70.939239
$630-645,77.821056,81.301007,70.623565,82.600247,58.841194
$645-680,77.049297,81.005604,66.230813,81.109397,53.528791


## Scores by School Size

* Perform the same operations as above, based on school size.

In [29]:
# Need data on school size for each student, math scores for each student, reading scores for each student (sort by ID)
studentsSpendingAndScores = student_data[["Student ID", "school_name", "reading_score", "math_score"]]
budgetPerStudent = school_data[["school_name","size"]]
studentsSpendingAndScores = pd.merge(studentsSpendingAndScores, budgetPerStudent, on="school_name")


# Create bin bounds
scoreBins = [0, 1000, 2000, 5000]


# Create bin labels
group_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]  


# Apply the bins to the data
pd.cut(studentsSpendingAndScores["size"], scoreBins, labels=group_labels).head()


# Add to the dataframe we're using
studentsSpendingAndScores["School Size"] = pd.cut(studentsSpendingAndScores["size"], scoreBins, labels=group_labels)


# Remove unneeded data
studentsSpendingAndScores = studentsSpendingAndScores[["reading_score","math_score","School Size"]]


# Organize the data by the bins (to get averages)
scoreByBudgetSum = studentsSpendingAndScores.groupby(["School Size"]).sum()

scoreByBudgetSize = studentsSpendingAndScores.groupby(["School Size"]).count()
scoreByBudgetSize = scoreByBudgetSize.rename(columns={"reading_score":"reading_size","math_score":"math_size"})

AverageScoresByBudget = pd.merge(scoreByBudgetSize, scoreByBudgetSum, on="School Size")


# Calculate averages and keep them
AverageScoresByBudget = AverageScoresByBudget.assign(average_math_score = (AverageScoresByBudget["math_score"]/AverageScoresByBudget["math_size"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(average_reading_score = (AverageScoresByBudget["reading_score"]/AverageScoresByBudget["reading_size"] ))
AverageScoresByBudget = AverageScoresByBudget[["average_math_score","average_reading_score"]]
AverageScoresByBudget = AverageScoresByBudget.rename(columns={"average_math_score":"Average Math Score","average_reading_score":"Average Reading Score"})


# Identify who is passing each category
passingMathListByBudget = studentsSpendingAndScores.loc[studentsSpendingAndScores["math_score"] >= 70]
passingReadingListByBudget = studentsSpendingAndScores.loc[studentsSpendingAndScores["reading_score"] >= 70]
passingBothListByBudget = passingMathListByBudget.loc[passingMathListByBudget["reading_score"] >= 70]


# Sum how many students are passing in each range
passingMathListByBudget = passingMathListByBudget.groupby(["School Size"]).count()
passingMathListByBudget = passingMathListByBudget[["reading_score"]]
passingMathListByBudget = passingMathListByBudget.rename(columns={"reading_score":"amount_passed_math"})

passingReadingListByBudget = passingReadingListByBudget.groupby(["School Size"]).count()
passingReadingListByBudget = passingReadingListByBudget[["reading_score"]]
passingReadingListByBudget = passingReadingListByBudget.rename(columns={"reading_score":"amount_passed_reading"})

passingBothListByBudget = passingBothListByBudget.groupby(["School Size"]).count()
passingBothListByBudget = passingBothListByBudget[["reading_score"]]
passingBothListByBudget = passingBothListByBudget.rename(columns={"reading_score":"amount_passed_both"})


# Count number of students from each range
countByRange = studentsSpendingAndScores.groupby(["School Size"]).count()
countByRange = countByRange[["reading_score"]]
countByRange = countByRange.rename(columns={"reading_score":"total_count"})


# Add necessary information to the chart
AverageScoresByBudget = pd.merge(AverageScoresByBudget, countByRange, on="School Size")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingMathListByBudget, on="School Size")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingReadingListByBudget, on="School Size")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingBothListByBudget, on="School Size")


# Calculate percentages of students passing in new columns
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_math = (100*AverageScoresByBudget["amount_passed_math"]/AverageScoresByBudget["total_count"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_reading = (100*AverageScoresByBudget["amount_passed_reading"]/AverageScoresByBudget["total_count"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_both = (100*AverageScoresByBudget["amount_passed_both"]/AverageScoresByBudget["total_count"] ))


# Re-name new columns
AverageScoresByBudget = AverageScoresByBudget.rename(columns={"percent_passed_math":"% Passing Math","percent_passed_reading":"% Passing Reading", "percent_passed_both":"% Overall Passing"})


# Keep only what is needed
AverageScoresByBudget = AverageScoresByBudget[["Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]


# Display result
AverageScoresByBudget



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


## Scores by School Type

* Perform the same operations as above, based on school type

In [30]:
# Need data on school type for each student, math scores for each student, reading scores for each student (sort by ID)
studentsSpendingAndScores = student_data[["Student ID", "school_name", "reading_score", "math_score"]]
budgetPerStudent = school_data[["school_name","type"]]
studentsSpendingAndScores = pd.merge(studentsSpendingAndScores, budgetPerStudent, on="school_name")


# Replace Charter with 1s and District with 3s to make binning easier
studentsSpendingAndScores["type"] = studentsSpendingAndScores["type"].replace(["Charter"],1)
studentsSpendingAndScores["type"] = studentsSpendingAndScores["type"].replace(["District"],3)


# Create bin bounds
scoreBins = [0, 2, 4]


# Create bin labels
group_labels = ["Charter", "District"]  


# Apply the bins to the data
pd.cut(studentsSpendingAndScores["type"], scoreBins, labels=group_labels).head()


# Add to the dataframe we're using
studentsSpendingAndScores["School Type"] = pd.cut(studentsSpendingAndScores["type"], scoreBins, labels=group_labels)


# Remove unneeded data
studentsSpendingAndScores = studentsSpendingAndScores[["reading_score","math_score","School Type"]]


# Organize the data by the bins (to get averages)
scoreByBudgetSum = studentsSpendingAndScores.groupby(["School Type"]).sum()

scoreByBudgetSize = studentsSpendingAndScores.groupby(["School Type"]).count()
scoreByBudgetSize = scoreByBudgetSize.rename(columns={"reading_score":"reading_size","math_score":"math_size"})

AverageScoresByBudget = pd.merge(scoreByBudgetSize, scoreByBudgetSum, on="School Type")


# Calculate averages and keep them
AverageScoresByBudget = AverageScoresByBudget.assign(average_math_score = (AverageScoresByBudget["math_score"]/AverageScoresByBudget["math_size"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(average_reading_score = (AverageScoresByBudget["reading_score"]/AverageScoresByBudget["reading_size"] ))
AverageScoresByBudget = AverageScoresByBudget[["average_math_score","average_reading_score"]]
AverageScoresByBudget = AverageScoresByBudget.rename(columns={"average_math_score":"Average Math Score","average_reading_score":"Average Reading Score"})


# Identify who is passing each category
passingMathListByBudget = studentsSpendingAndScores.loc[studentsSpendingAndScores["math_score"] >= 70]
passingReadingListByBudget = studentsSpendingAndScores.loc[studentsSpendingAndScores["reading_score"] >= 70]
passingBothListByBudget = passingMathListByBudget.loc[passingMathListByBudget["reading_score"] >= 70]


# Sum how many students are passing in each range
passingMathListByBudget = passingMathListByBudget.groupby(["School Type"]).count()
passingMathListByBudget = passingMathListByBudget[["reading_score"]]
passingMathListByBudget = passingMathListByBudget.rename(columns={"reading_score":"amount_passed_math"})

passingReadingListByBudget = passingReadingListByBudget.groupby(["School Type"]).count()
passingReadingListByBudget = passingReadingListByBudget[["reading_score"]]
passingReadingListByBudget = passingReadingListByBudget.rename(columns={"reading_score":"amount_passed_reading"})

passingBothListByBudget = passingBothListByBudget.groupby(["School Type"]).count()
passingBothListByBudget = passingBothListByBudget[["reading_score"]]
passingBothListByBudget = passingBothListByBudget.rename(columns={"reading_score":"amount_passed_both"})


# Count number of students from each range
countByRange = studentsSpendingAndScores.groupby(["School Type"]).count()
countByRange = countByRange[["reading_score"]]
countByRange = countByRange.rename(columns={"reading_score":"total_count"})


# Add necessary information to the chart
AverageScoresByBudget = pd.merge(AverageScoresByBudget, countByRange, on="School Type")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingMathListByBudget, on="School Type")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingReadingListByBudget, on="School Type")
AverageScoresByBudget = pd.merge(AverageScoresByBudget, passingBothListByBudget, on="School Type")


# Calculate percentages of students passing in new columns
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_math = (100*AverageScoresByBudget["amount_passed_math"]/AverageScoresByBudget["total_count"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_reading = (100*AverageScoresByBudget["amount_passed_reading"]/AverageScoresByBudget["total_count"] ))
AverageScoresByBudget = AverageScoresByBudget.assign(percent_passed_both = (100*AverageScoresByBudget["amount_passed_both"]/AverageScoresByBudget["total_count"] ))


# Re-name new columns
AverageScoresByBudget = AverageScoresByBudget.rename(columns={"percent_passed_math":"% Passing Math","percent_passed_reading":"% Passing Reading", "percent_passed_both":"% Overall Passing"})


# Keep only what is needed
AverageScoresByBudget = AverageScoresByBudget[["Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]


# Display result
AverageScoresByBudget


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
